<a href="https://colab.research.google.com/github/manishkumarray/COVID-19-Detection-on-X-Ray-Images/blob/main/COVID_19_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Name: Manish Kumar Ray**

# **Eno.: 18012011041**

# **Batch: DL-1**

## **Project: COVID-19 Detection on X-Ray Images**

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master.zip" -d "/content/drive/MyDrive/Colab Notebooks/dl dataset/"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image

## **Buiding the CNN Architecture**

In [ ]:
# CNN Based Model in Keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

## **Data Augmentation, Normalizing Data & Training the Model**

In [ ]:
# Train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

In [ ]:
train_generator.class_indices

In [ ]:
validation_generator = test_dataset.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=7,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=2
)

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/model_adv.h5")

In [ ]:
model.evaluate_generator(train_generator)

In [ ]:
model.evaluate_generator(validation_generator)

## **Model Evaluation**

In [ ]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/model_adv.h5')

In [ ]:
import os

In [ ]:
train_generator.class_indices

In [ ]:
y_actual = []
y_test = []

In [ ]:
from PIL import Image
for i in os.listdir("/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/Val/Normal/"):
  img = image.load_img("/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/Val/Normal/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict(img)
  y_test.append(p[0,0])
  y_actual.append(1)

## **Confusion Matrix**

In [ ]:
y_actual = np.array(y_actual)
y_test = np.array(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_actual, y_test)

In [ ]:
import seaborn as sns
sns.heatmap(cm, cmap=None, annot=True)

## **Plotting Training & Validation Loss**

In [ ]:
import matplotlib.pyplot as plt
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## **Plotting Training & Validation Accuracy**

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
plt.plot(epochs, acc, color='red', label='Training acc')
plt.plot(epochs, val_acc, color='green', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## **Some Comparative Results of Radiologist vs CNN Model on Validation set**

In [ ]:
class color:
   GREEN = '\033[92m'
   RED = '\033[91m'
   
from PIL import Image
j = 0
for i in os.listdir("/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/Val/Normal/"):
  j = j+1
  pil_img = Image.open('/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/Val/Normal/'+i)
  im = pil_img
  size = (200,200) #image size
  
  #resize image
  out = im.resize(size)
  display(out)
  img = image.load_img("/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/Val/Normal/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict(img)
  print(color.GREEN + 'Radiologist: COVID-19 -ve' + color.GREEN)

  if p==0:
    print(color.RED + 'CNN Model: COVID-19 +ve' + color.RED)
  else:
    print("CNN Model: COVID-19 -ve")
  if (j==5):
    break

from PIL import Image
j = 0
for i in os.listdir("/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/Val/Covid/"):
  j = j+1
  pil_img = Image.open('/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/Val/Covid/'+i)
  im = pil_img
  size = (200,200) #image size
  
  #resize image
  out = im.resize(size)
  display(out)
  img = image.load_img("/content/drive/MyDrive/Colab Notebooks/dl dataset/COVID-19-master/Val/Covid/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict(img)
  print(color.RED + 'Radiologist: COVID-19 +ve' + color.RED)
  if p==0:
    print("CNN Model: COVID-19 +ve")
  else:
    print(color.GREEN + 'Radiologist: COVID-19 -ve' + color.GREEN)
  if (j==5):
    break